<p style="text-align:center">
    <a href="https://www.ptit.edu.vn" target="_blank">
        <img src="https://i.imgur.com/XBSxCH1.png" width="400" alt="PTIT Logo">
    </a>
</p>


<center>

# Practice lab: Cardiovascular Prediction

</center>


Reference: https://www.sciencedirect.com/science/article/abs/pii/S0957417420302323



### MỤC TIÊU VÀ LÝ DO NGHIÊN CỨU

- Mục tiêu nghiên cứu: Nghiên cứu này nhằm phát hiện một mạng nơ-ron tích chập để dự đoán **bệnh tim mạch vành (CHD)** dựa trên dữ liệu sẵn có của **NHANES**, nghiên cứu tập trung vào cải thiện độ chính xác trong việc chẩn đoán **CHD** 
- Lý do: **Bệnh tim mạch vành** là một trong những nguyên nhân chính gây tử vong trên toàn thế giới, việc chẩn đoán sớm có thể giúp giảm nguy cơ và phòng ngừa các cơn đau tim. Tuy nhiên việc sử dụng các mô hình học máy, học sâu truyền thống thường gặp khó khăn khi áp dụng bộ dữ liệu này

### PHƯƠNG PHÁP NGHIÊN CỨU

- Bộ dữ liệu bao gồm thông tin lâm sàng và nhân khẩu học của 37079 cá nhân (1300 ca CHD và 35779 ca Non-CHD)
- Sử dụng các mô hình học máy như Random Forest, SVM và học sâu CNN để thử với bộ dữ liệu trên

### THƯ VIỆN CƠ BẢN

In [39]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score

### ĐỌC DỮ LIỆU TỪ DATASET

In [19]:
file_path = r'C:\Users\baoho\Desktop\PTIT-CNN-Prediction-Cardiovascular\CardiacPrediction.xlsx'
df = pd.read_excel(file_path)

### KHAI PHÁ DỮ LIỆU

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37079 entries, 0 to 37078
Data columns (total 51 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SEQN                         37079 non-null  int64  
 1   Gender                       37079 non-null  int64  
 2   Age                          37079 non-null  int64  
 3   Annual-Family-Income         37079 non-null  int64  
 4   Ratio-Family-Income-Poverty  37079 non-null  float64
 5   X60-sec-pulse                37079 non-null  int64  
 6   Systolic                     37079 non-null  int64  
 7   Diastolic                    37079 non-null  int64  
 8   Weight                       37079 non-null  float64
 9   Height                       37079 non-null  float64
 10  Body-Mass-Index              37079 non-null  float64
 11  White-Blood-Cells            37079 non-null  float64
 12  Lymphocyte                   37079 non-null  float64
 13  Monocyte        

In [21]:
# I want to see std, min25%, min50%, min75%, max of each column
df.describe()

,SEQN,Gender,Age,Annual-Family-Income,Ratio-Family-Income-Poverty,X60-sec-pulse,Systolic,Diastolic,Weight,Height,...,Total-Cholesterol,HDL,Glycohemoglobin,Vigorous-work,Moderate-work,Health-Insurance,Diabetes,Blood-Rel-Diabetes,Blood-Rel-Stroke,CoronaryHeartDisease
count,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,...,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000,37079.000000
mean,48901.041236,1.513282,48.943661,7.358208,2.559026,72.579250,124.090078,69.919253,80.988276,167.389601,...,5.081713,1.370344,5.676496,1.783840,1.598856,1.218587,1.907333,1.549502,1.796165,0.040670
std,26753.636441,0.499830,18.010440,3.994083,1.624789,12.242108,19.254741,13.575804,20.678734,10.122908,...,1.072682,0.415985,1.050223,0.448324,0.511199,0.461102,0.349674,0.497550,0.402853,0.197527
min,2.000000,1.000000,20.000000,1.000000,0.000000,32.000000,0.000000,0.000000,32.300000,129.700000,...,1.530000,0.160000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,26120.500000,1.000000,33.000000,4.000000,1.140000,64.000000,111.000000,62.000000,66.500000,160.000000,...,4.320000,1.070000,5.200000,2.000000,1.000000,1.000000,2.000000,1.000000,2.000000,0.000000
50%,50065.000000,2.000000,48.000000,7.000000,2.180000,72.000000,121.000000,70.000000,78.200000,167.100000,...,5.020000,1.290000,5.400000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
75%,71173.500000,2.000000,63.000000,10.000000,4.130000,80.000000,134.000000,78.000000,92.100000,174.600000,...,5.740000,1.600000,5.800000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,0.000000
max,93702.000000,2.000000,85.000000,15.000000,5.000000,224.000000,270.000000,132.000000,371.000000,204.500000,...,14.090000,5.840000,18.800000,3.000000,3.000000,9.000000,3.000000,2.000000,2.000000,1.000000


### GIẢI THÍCH BỘ DỮ LIỆU

1. Nhân khẩu học (Demographic)
- Age: Tuổi của người tham gia khảo sát
- Gender: Giới tính (1 - Nam, 2 - Nữ)
2. Biến số kiểm tra (Examination Variables)
- Height: Chiều cao (cm)
- Weight: Cân nặng (kg)
- Huyết áp (Systolic/Diastolic): Huyết áp tâm thu và huyết áp tâm trương là 2 chỉ số quan trọng để chẩn đoán bệnh nhân mắc bệnh tim mạch
- BMI: Chỉ số khối cơ thể, những người có chỉ số BMI cao thường dễ mắc các bệnh tim mạch hơn bình thường
3. Biến số thí nghiệm (Laboratory Variables)
- Mức độ Choresterol: Ba gồm các chỉ số Choresterol toàn phần như LDL (Low-Density Lipprotein) và HDL (High-Density Lipprotein). Choresterol cao là một trong những nguyên nhân hàng đầu gây ra bệnh tim mạch
- Glucose: Mức đường trong máu, liên quan mật thiết đến bệnh tiểu đường, một yếu tố nguy hiểm đối với người bệnh tim
- Triglycerides: Mức chất béo trong máu
- Serum Albumin, Lactate Dehydrogenase (LDH): Chỉ số sinh hóa máu liên quan đến sức khỏe tổng quát và nguy cơ mắc bệnh tim mạch
4. Biến số khảo sát (Questionare Variables)
- Tình trạng hút thuốc lá
- Tình trạng bệnh tiểu đường
- Physical Activity (Hoạt động thể chất): Hoạt động thể chất mạnh, vừa phải có liên quan đến bệnh tim mạch
5. Biến số độc lập
- CoronaryHeartDisease: Biến nhị phân biểu thị việc người tham gia khảo sát có mắc bệnh tim mạch vành không (1 - Có, 2 - Không)

In [22]:
# I want to know how many 0 and how many 1 in column CHD
df['CoronaryHeartDisease'].value_counts()

CoronaryHeartDisease
0    35571
1     1508
Name: count, dtype: int64

📶  Có thể thấy sự mất cân bằng dữ liệu rất lớn trên tập dữ liệu này với tỉ lệ 1:35 (Người mắc bệnh : Người không mắc bệnh)

### TIỀN XỬ LÝ DỮ LIỆU

Kiểm tra các giá trị khuyết (Null, NaN)

In [23]:
null_counts = df.isnull().sum()
print(null_counts)



SEQN                           0
Gender                         0
Age                            0
Annual-Family-Income           0
Ratio-Family-Income-Poverty    0
X60-sec-pulse                  0
Systolic                       0
Diastolic                      0
Weight                         0
Height                         0
Body-Mass-Index                0
White-Blood-Cells              0
Lymphocyte                     0
Monocyte                       0
Eosinophils                    0
Basophils                      0
Red-Blood-Cells                0
Hemoglobin                     0
Mean-Cell-Vol                  0
Mean-Cell-Hgb-Conc.            0
Mean-cell-Hemoglobin           0
Platelet-count                 0
Mean-Platelet-Vol              0
Segmented-Neutrophils          0
Hematocrit                     0
Red-Cell-Distribution-Width    0
Albumin                        0
ALP                            0
AST                            0
ALT                            0
Cholestero

❌ Không có giá trị NULL trong bộ dữ liệu 

Kiểm tra trùng lặp

In [24]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")


Number of duplicate rows: 0


❌ Không có giá trị trùng lặp trong bộ dữ liệu 

### CHUẨN HÓA DỮ LIỆU

Chuẩn hóa dữ liệu là một bước quan trọng trong quá trình xử lý dữ liệu nhằm đảm bảo các biến số có cùng đơn vị và tỉ lệ, giúp các mô hình học máy hoạt động tốt hơn. Dưới đây là một số lý do chính:
- Giảm sự thiên lệch giữa các biến: Các biến trong bộ dữ liệu có đơn vị đo lường khác nhau, ví dụ cân nặng dao động từ 50-100kg, trong khi mức Choresterol thường nằm trong khoảng 1-10 mmol/L. Điều này khiến các biến có giá trị lớn hơn ảnh hưởng mạnh hơn đến mô hình, gây mất cân bằng so với các trọng số khác
- Cải thiện tốc độ hội tụ: Trong các mô hình như mạng nơ-ron, việc chuẩn hóa các biến có thể cải thiện tốc độ huấn luyện và hiệu quả của các thuật toán tối ưu hóa như Gradient Descent - bằng cách đưa các biến về chung một thang đo
- Hỗ trợ các mô hình dựa trên khoảng cách: Với các mô hình như KNN hoặc SVM, khoảng cách giữa các điểm dữ liệu rất quan trọng. Nếu các biến có thang đo khác nhau, các biến lớn hơn sẽ chi phối khoảng cách giữa các điểm dữ liệu, dẫn đến sai lệch nghiêm trọng kết quả đầu ra

🎯 Sử dụng Kỹ thuật Standardization (Chuẩn hóa theo Z-score). Phương pháp này sẽ chuyển đổi các giá trị để chúng có giá trị trung bình là 0 và độ lệch chuẩn là 1

In [25]:


#List columns to be scaled
columns_to_standardize = [
    'Age', 'Ratio-Family-Income-Poverty', 'X60-sec-pulse', 'Systolic', 'Diastolic',
    'Weight', 'Height', 'Total-Cholesterol', 'HDL', 'Glycohemoglobin'
]
# Initialize the StandardScaler
scaler = StandardScaler()

# Apply standardization to the selected columns
df[columns_to_standardize] = scaler.fit_transform(df[columns_to_standardize])

# Display the first few rows after standardization
df[columns_to_standardize].head()

,Age,Ratio-Family-Income-Poverty,X60-sec-pulse,Systolic,Diastolic,Weight,Height,Total-Cholesterol,HDL,Glycohemoglobin
0,1.557803,1.502353,-0.374062,-1.355013,-1.025312,-0.270246,0.653023,0.445885,0.047252,-0.929811
1,0.003128,1.502353,-0.537435,-0.108550,0.963547,0.556701,1.077808,1.984107,-0.697977,-0.168058
2,-0.663161,1.459270,-0.700808,2.592120,2.142130,0.880710,1.245746,-0.980466,-0.938374,-0.453715
3,1.169134,-0.916455,2.403275,0.306937,-0.288698,-0.840889,-0.957208,2.832459,-0.217184,1.831544
4,1.779899,0.068302,-0.047317,0.618553,-0.657005,-0.265410,-0.117517,-0.542305,-0.794136,0.117599


Dữ liệu đã được chuẩn hóa thành công, các biến liên tục như tuổi, tỉ lệ thu nhập, huyết áp, cân nặng, chiều cao và các chỉ số liên quan đến choresterol, đường huyết học đã được đưa về thang đo có trung bình bằng 0 và độ lệch chuẩn  bằng 1. Điều này giúp các mô hình học máy xử lý dữ liệu một cách nhất quán và giảm thiểu sự thiên lệch do khác biệt đơn vị gây ra

### CHỌN LỌC BIẾN (FEATURES SELECTION)

Định nghĩa Tập dữ liệu và biến mục tiêu
- X: Tập hợp các biến độc lập của tập dữ liệu, loại bỏ cột SEQN vì không có giá trị phân tích và cột ConoraryHeartDisease - biến mục tiêu
- y : Biến mục tiêu, biểu thị cá nhân có mắc bệnh tim mạch vành không 

In [26]:
X = df.drop(columns = ['SEQN', 'CoronaryHeartDisease'])
y = df['CoronaryHeartDisease']

Khởi tạo mô hình Lasso kết hợp Cross-Validation, trong đó cv = 5 tức là Lasso sẽ chia tập dữ liệu thành 5 phần, và thực hiện 5 lần kiểm tra chéo. Giúp mô hình tự động chọn tham số điều chỉnh alpha tối ưu để điều chỉnh mức phạt (shrinkage) cho các hệ số

In [27]:


lasso = LassoCV(cv = 5, random_state = 42)

# Fit the model to data
lasso.fit(X, y)


LassoCV(cv=5, random_state=42)

Lấy các hệ số của các biến sau khi huấn luyện:
- lasso.coef_: Sau khi huấn luyện, mô hình LASSO trả về các hệ số hồi quy tương ứng với từng biến trong tập X. Nếu hệ số của một biến bằng 0, điều đó có nghĩa biến đó có đóng góp không đáng kể cho mô hình
- pd.Series(lasso.coef_, index = X.columns): Tạo một Series pandas để lưu trữ các hệ số hồi quy, trong đó tên của các biến được đặt làm chỉ mục

In [28]:
lasso_coefficients = pd.Series(lasso.coef_, index=X.columns)


Lọc các biến quan trọng với hệ số khác 0

In [29]:
selected_features = lasso_coefficients[lasso_coefficients != 0]


Hiển thị các biến đã chọn

In [30]:
selected_features


Age                            0.032559
Annual-Family-Income          -0.000924
X60-sec-pulse                 -0.006702
Systolic                      -0.002680
Diastolic                     -0.004365
Body-Mass-Index               -0.000158
White-Blood-Cells              0.001968
Lymphocyte                    -0.001142
Monocyte                       0.002336
Eosinophils                    0.000366
Mean-Cell-Vol                  0.000891
Platelet-count                -0.000127
Mean-Platelet-Vol             -0.000935
Hematocrit                    -0.000743
Red-Cell-Distribution-Width    0.004666
Albumin                       -0.000069
ALP                           -0.000010
AST                           -0.000221
ALT                            0.000062
Cholesterol                   -0.016272
Creatinine                     0.000183
Glucose                        0.000694
GGT                            0.000003
Iron                          -0.000073
LDH                            0.000193


Dựa trên hồi quy LASSO, các biến quan trọng được chọn lọc có hệ số khác 0. Những biến này là biến quan trọng có khả năng ảnh hưởng đến khả năng dự đoán bệnh tim mạch vành (CHD). Một số biến đáng chú ý bao gồm:
- Age: Có ảnh hưởng (+) đến người mắc bệnh CHD
- Systolic, Diasotic (Huyết áp): Có ảnh hưởng (-), nhưng mức độ nhỏ
- Cholesterol, HDL: Choresterol có ảnh hưởng âm mạnh hơn, trong khi HDL cũng có tác động âm nhẹ
- Triglycerides và Glycohemoglobin: Có ảnh hưởng (+)
- Blood-Rel-Stroke (Tiền sử đột quỵ): Có ảnh hưởng tiêu cực (-)

### HUẤN LUYỆN MÔ HÌNH HỒI QUY LOGICSTIC

- max_iter = 1000: Tham số giới hạn lần lặp trong quá trình tối ưu hóa, thông số đảm bảo mô hình sẽ dừng lại sau 1000 lần lặp nếu mô hình chưa hội tụ
- random_state = 42: Cố định các trạng thái ngẫu nhiên

In [31]:


# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

logistic_model = LogisticRegression(max_iter=1000, random_state=42)


In [32]:
logistic_model.fit(X_train, y_train)



c:\Users\baoho\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000, random_state=42)

In [33]:
# Make predictions on the testing data
y_pred = logistic_model.predict(X_test)
y_proba = logistic_model.predict_proba(X_test)[:, 1]

In [34]:


# Calculate performance metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc_roc = roc_auc_score(y_test, y_proba)


In [35]:
# Display the performance metrics
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'AUC-ROC'],
    'Score': [accuracy, recall, auc_roc]
})

# Show the performance metrics
performance_metrics

,Metric,Score
0,Accuracy,0.959457
1,Recall,0.033259
2,AUC-ROC,0.874843


THAM SỐ ĐÁNH GIÁ
- Accuracy: Giá trị đạt 95,94% cho thấy mô hình dự đoán chính xác phần lớn các mẫu dữ liệu
- Recall: Giá trị 3,3% là quá thấp, có nghĩa mô hình chỉ nhận diện được 3,3% các ca mắc bệnh. Cho thấy mô hình không đủ tốt
- AUC-ROC: Giá trị 87,48% cho thấy khả năng phân biệt giữa các lớp vẫn khá tốt. Có nghĩa là mô hình có khả năng dự đoán tốt khi so sánh các xác suất dự đoán giữa 2 lớp nhưng kết quả Recall thấp cho thấy mô hình chủ yếu tập trung vào dự đoán các ca không mắc bệnh

Nguyên nhân: Do sự mất cân bằng dữ liệu quá lớn, khiến các mô hình chỉ dự đoán các trường hợp không mắc bệnh nhằm đạt độ chính xác cao

GIẢI PHÁP
- Điều chỉnh trọng số lớp (class weights)
- Undersampling hoặc Oversampling để cân bằng dữ liệu
- Sử dụng các thuật toán học máy, học sâu khác

In [36]:
# Initialize Logistic Regression model with class_weight='balanced'
logistic_model_balanced = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

# Fit the model on the training data
logistic_model_balanced.fit(X_train, y_train)

# Make predictions on the testing data
y_pred_balanced = logistic_model_balanced.predict(X_test)
y_proba_balanced = logistic_model_balanced.predict_proba(X_test)[:, 1]

# Calculate performance metrics
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
recall_balanced = recall_score(y_test, y_pred_balanced)
auc_roc_balanced = roc_auc_score(y_test, y_proba_balanced)

# Display the performance metrics
performance_metrics_balanced = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'AUC-ROC'],
    'Score': [accuracy_balanced, recall_balanced, auc_roc_balanced]
})

performance_metrics_balanced

c:\Users\baoho\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,Metric,Score
0,Accuracy,0.772204
1,Recall,0.840355
2,AUC-ROC,0.876630


### NHẬN XÉT
- Trước khi điều chỉnh trọng số lớp cân bằng
  - Accuracy (Độ chính xác): Đạt 95,59% Mô hình đạt độ chính xác cao, nhưng chủ yếu đến từ các trường hợp không mắc bệnh (do lớp này chiếm ưu thế)
  - Recall (Độ nhạy): Đạt 3.33% - Giá trị rất thấp, cho thấy mô hình bỏ qua phần lớn các trường hợp mắc bệnh, dẫn đến việc phát hiện các ca dương tính là không đáng kể
  - AUC-ROC: Đạt 87,48% - Cho thấy mô hình vẫn có thể phân biệt giữa 2 lớp, nhưng hiệu suất không thể hiện tốt ở việc phát hiện đúng các trường hợp dương tính
- Sau khi điều chỉnh trọng số lớp cân bằng
  - Accuracy (Độ chính xác): 77,22% - Mô hình đạt độ chính xác khá
  - Recall (Độ nhạy): Độ nhạy tăng đáng kể lên đến 84,03% cho thấy mô hình đã có thể phát hiện phần lớn các trường hợp mắc bệnh
  - AUC-ROC: Đạt 87,66% tăng nhẹ so với khi chưa dùng trọng số lớp, không đáng kể


Ý NGHĨA CỦA SỰ KHÁC BIỆT
- Trước khi điều chỉnh trọng số: Mô hình chủ yếu tập trung vào dự đoán các trường hợp không mắc bệnh để đạt độ chính xác cao. Tuy nhiên điều này làm mất đi khả năng phát hiện các trường hợp dương tính, khiến mô hình không phù hợp cho dự đoán các ca bệnh, đặc biệt trong bối cảnh sức khỏe
- Sau khi điều chỉnh trọng số: Mô hình điều chỉnh trọng số để đối xử cân bằng hơn giữa các lớp. Mặc dù độ chính xác tổng thể giảm, mô hình có thể phát hiện hầu hết các ca mắc bệnh, làm cho nó trở nên hữu ích hơn trong ứng dụng thực tế

In [40]:
# Initialize Random Forest model without class weights
random_forest_model_no_weights = RandomForestClassifier(random_state=42)

# Fit the model on the training data
random_forest_model_no_weights.fit(X_train, y_train)

# Make predictions on the testing data
y_pred_rf_no_weights = random_forest_model_no_weights.predict(X_test)
y_proba_rf_no_weights = random_forest_model_no_weights.predict_proba(X_test)[:, 1]

# Calculate performance metrics without class weights
accuracy_rf_no_weights = accuracy_score(y_test, y_pred_rf_no_weights)
recall_rf_no_weights = recall_score(y_test, y_pred_rf_no_weights)
auc_roc_rf_no_weights = roc_auc_score(y_test, y_proba_rf_no_weights)

# Store performance results without class weights
rf_performance_no_weights = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'AUC-ROC'],
    'Score (No Weights)': [accuracy_rf_no_weights, recall_rf_no_weights, auc_roc_rf_no_weights]
})

# Now, initialize Random Forest model with class weights balanced
random_forest_model_balanced = RandomForestClassifier(random_state=42, class_weight='balanced')

# Fit the model on the training data
random_forest_model_balanced.fit(X_train, y_train)

# Make predictions on the testing data
y_pred_rf_balanced = random_forest_model_balanced.predict(X_test)
y_proba_rf_balanced = random_forest_model_balanced.predict_proba(X_test)[:, 1]

# Calculate performance metrics with class weights balanced
accuracy_rf_balanced = accuracy_score(y_test, y_pred_rf_balanced)
recall_rf_balanced = recall_score(y_test, y_pred_rf_balanced)
auc_roc_rf_balanced = roc_auc_score(y_test, y_proba_rf_balanced)

# Store performance results with class weights balanced
rf_performance_balanced = pd.DataFrame({
    'Metric': ['Accuracy', 'Recall', 'AUC-ROC'],
    'Score (Balanced Weights)': [accuracy_rf_balanced, recall_rf_balanced, auc_roc_rf_balanced]
})

# Combine both results for comparison
rf_comparison = pd.merge(rf_performance_no_weights, rf_performance_balanced, on='Metric')

# Display the comparison of results
rf_comparison

,Metric,Score (No Weights),Score (Balanced Weights)
0,Accuracy,0.959457,0.959187
1,Recall,0.000000,0.000000
2,AUC-ROC,0.853441,0.862268
